# HW 3: Key
* If you replace the import of the key transforms file with yours, the outputs in each cell below shouldn't change if your code is correct. 

In [1]:
import sys
sys.path.append("/home/daniel/Documents/HW")
import transforms as tr
import numpy as np
from visualization import VizScene 

# Problem # 1
### Homogeneous matrix made of only rotation about z-axis by amount $\frac{\pi}{4}$

Rotation by amount theta around the z-axis should give the following:
$$
\left[\begin{matrix}0.707 & -0.707 & 0 & 0\\ 0.707 & 0.707 & 0 & 0\\0 & 0 & 1 & 0\\0 & 0 & 0 & 1\end{matrix}\right]
$$

In [2]:
T = tr.se3(tr.rotz(np.pi/4))
print(T)


[[ 0.70710678 -0.70710678  0.          0.        ]
 [ 0.70710678  0.70710678  0.          0.        ]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          0.          1.        ]]


### Translation only along the x-axis by an amount $0.5$ should give the following:
$$
\left[\begin{matrix}1 & 0 & 0 & 0.5 \\0 & 1 & 0 & 0\\0 & 0 & 1 & 0\\0 & 0 & 0 & 1\end{matrix}\right]
$$

In [3]:
T = tr.se3(p=[0.5, 0, 0])
print(T)

[[1.  0.  0.  0.5]
 [0.  1.  0.  0. ]
 [0.  0.  1.  0. ]
 [0.  0.  0.  1. ]]


### Checking the "inv" function

If we calculate a homogenous transform with a rotation of 45 degrees about the x-axis and a general translation ($[0.5, 0.25, 0.3]^T$) we get the following:
$$
\left[\begin{matrix}1 & 0 & 0 & 0.5\\0 & 0.707106781186548 & -0.707106781186547 & 0.25\\0 & 0.707106781186547 & 0.707106781186548 & 0.3\\0 & 0 & 0 & 1\end{matrix}\right]
$$

Then, we can multiply the original transform by its inverse to check out inverse function, we should get the following:
$$
\left[\begin{matrix}1 & 0 & 0 & 0\\0 & 1.0 & 0 & 0\\0 & 0 & 1.0 & 0\\0 & 0 & 0 & 1\end{matrix}\right]
$$

In [4]:
T = tr.se3(tr.rotx(np.pi/4), [0.5, 0.25, 0.3])
print(T)

# now we can check if we implemented "inv" correctly:
check = tr.inv(T) @ T
print("\n\n inv(T) @ T should give identity matrix:")
print(check)


[[ 1.          0.          0.          0.5       ]
 [ 0.          0.70710678 -0.70710678  0.25      ]
 [ 0.          0.70710678  0.70710678  0.3       ]
 [ 0.          0.          0.          1.        ]]


 inv(T) @ T should give identity matrix:
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00 -1.01465364e-17  0.00000000e+00]
 [ 0.00000000e+00 -1.01465364e-17  1.00000000e+00  6.93889390e-18]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


### Using DH parameters to find A matrices (for each joint):

Remember that if we combine a rotation in z, translation in z, then translation in x, and rotation in x, we should get the same result as the book for following the DH convention to move from one frame (or joint) to another as follows: 

$$
A = \left[\begin{matrix}\cos{\left(\theta \right)} & - \sin{\left(\theta \right)} \cos{\left(\alpha \right)} & \sin{\left(\alpha \right)} \sin{\left(\theta \right)} & a \cos{\left(\theta \right)}\\\sin{\left(\theta \right)} & \cos{\left(\alpha \right)} \cos{\left(\theta \right)} & - \sin{\left(\alpha \right)} \cos{\left(\theta \right)} & a \sin{\left(\theta \right)}\\0 & \sin{\left(\alpha \right)} & \cos{\left(\alpha \right)} & d\\0 & 0 & 0 & 1\end{matrix}\right]
$$

In future homework, we'll implement representations of robot arms that include this transformation. But for the test values of DH parameters below, we can perform this sequence of operations manually.  

Assuming the following DH parameters for two joints:

$\theta_1 = \frac{\pi}{8}$, $d_1 = 0$, $a_1 = 0.3$, $\alpha_1 = \frac{\pi}{2}$

$\theta_2 = \frac{\pi}{4}$, $d_2 = 0$, $a_2 = 0.3$, $\alpha_2 = 0$


The resulting homogeneous transform describing the tip (or frame 2) relate to frame 0 would be:

$$
\left[\begin{matrix}0.653281482438188 & -0.653281482438188 & 0.38268343236509 & 0.473148304484842\\0.270598050073099 & -0.270598050073099 & -0.923879532511287 & 0.195984444731456\\0.707106781186548 & 0.707106781186548 & 0 & 0.212132034355964\\0 & 0 & 0 & 1.0\end{matrix}\right]
$$

In [5]:
# start by substituting the actual values for R and p and making a new SE3 object
# that describes the transformation from frame 0 to frame 1

# find the transformation from frame 0 to 1
T1_in_0 = tr.se3(R=tr.rotz(np.pi/8.0))\
    @tr.se3(p = np.array([0, 0, 0]))\
    @tr.se3(p = np.array([0.3, 0, 0]))\
    @tr.se3(R=tr.rotx(np.pi/2))

# do the same thing for frame 1 to frame 2
T2_in_1 = tr.se3(R=tr.rotz(np.pi/4.0))\
    @tr.se3(p = np.array([0, 0, 0]))\
    @tr.se3(p = np.array([0.3, 0, 0]))\
    @tr.se3(R=tr.rotx(0.0))

# now we can combine the two to get a transformation that describes frame 2 
# relative to frame 0
T2_in_0 = T1_in_0 @ T2_in_1


# printing the result
print(T2_in_0)

# use the "add_frame" function to plot both frames (for joint 1 and joint 2) relative 
# to a base or ground frame. 

from visualization import VizScene 
import time
                         
viz = VizScene()
viz.add_frame(np.eye(4), label='world')
viz.add_frame(T1_in_0, label='T1_in_0')
viz.add_frame(T2_in_0, label='T2_in_0')

time_to_run = 15
refresh_rate = 60
for i in range(refresh_rate * time_to_run):
    viz.update()
    time.sleep(1.0/refresh_rate)
    
viz.close_viz()

[[ 6.53281482e-01 -6.53281482e-01  3.82683432e-01  4.73148304e-01]
 [ 2.70598050e-01 -2.70598050e-01 -9.23879533e-01  1.95984445e-01]
 [ 7.07106781e-01  7.07106781e-01  6.12323400e-17  2.12132034e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


/home/daniel/.local/lib/python3.8/site-packages/pyqtgraph/opengl/GLViewWidget.py:260: RuntimeWarning: 
Traceback (most recent call last):
  File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/daniel/.local/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/daniel/.local/lib/python3.8/site-packages/traitlets/config/application.py", line 976, in launch_instance
    app.start()
  File "/home/daniel/.local/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/home/daniel/.local/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 215, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.8/asyncio/base_events.py", line 570, in run_forever
    self._run_once()
  File "/usr/lib/python3.8

Error while drawing item <visualization.GLTextItem object at 0x7fe2bc65a5e0>.
Error while drawing item <visualization.GLTextItem object at 0x7fe2bc65a700>.
Error while drawing item <visualization.GLTextItem object at 0x7fe2bc65a820>.
Error while drawing item <visualization.GLTextItem object at 0x7fe2bc65a5e0>.
Error while drawing item <visualization.GLTextItem object at 0x7fe2bc65a700>.
Error while drawing item <visualization.GLTextItem object at 0x7fe2bc65a820>.
Error while drawing item <visualization.GLTextItem object at 0x7fe2bc65a5e0>.
Error while drawing item <visualization.GLTextItem object at 0x7fe2bc65a700>.
Error while drawing item <visualization.GLTextItem object at 0x7fe2bc65a820>.


# Problem 2 (a) and (b)

For each of the homogenous transforms below, I determined them by inspection. Also the variable "T1_in_0" is short for T describing frame 1 relative to frame 0, or $$T_1^0$$

In [7]:
# I determined all of the following by inspection (describing the coordinate frame relative to the other frame). 
T1_in_0 = tr.se3(p=[0, 1, 1])
T2_in_0 = tr.se3(p=[-0.5, 1.5, 1.1])
T3_in_0 = tr.se3(R=np.array([[0, 1, 0],[1, 0, 0],[0, 0, -1]]), p=[-0.5, 1.5, 3])

# now we can calculate T2_to_3 (but getting it by inspection is OK too)
T3_in_2 = tr.inv(T2_in_0) @ T3_in_0

# printing the results
print(T1_in_0)
print(T2_in_0)
print(T3_in_0)
print(T3_in_2)

# sending all the frames to our visualization
viz = VizScene()
viz.add_frame(np.eye(4), label='world')
viz.add_frame(T1_in_0, label='T1_in_0')
viz.add_frame(T2_in_0, label='T2_in_0')
viz.add_frame(T3_in_0, label='T3_in_0')

time_to_run = 30
refresh_rate = 60
for i in range(refresh_rate * time_to_run):
    viz.update()
    time.sleep(1.0/refresh_rate)
    
viz.close_viz()


[[1. 0. 0. 0.]
 [0. 1. 0. 1.]
 [0. 0. 1. 1.]
 [0. 0. 0. 1.]]
[[ 1.   0.   0.  -0.5]
 [ 0.   1.   0.   1.5]
 [ 0.   0.   1.   1.1]
 [ 0.   0.   0.   1. ]]
[[ 0.   1.   0.  -0.5]
 [ 1.   0.   0.   1.5]
 [ 0.   0.  -1.   3. ]
 [ 0.   0.   0.   1. ]]
[[ 0.   1.   0.   0. ]
 [ 1.   0.   0.   0. ]
 [ 0.   0.  -1.   1.9]
 [ 0.   0.   0.   1. ]]
Error while drawing item <visualization.GLTextItem object at 0x7fe2a521b550>.
Error while drawing item <visualization.GLTextItem object at 0x7fe2a521ba60>.
Error while drawing item <visualization.GLTextItem object at 0x7fe2a521baf0>.
Error while drawing item <visualization.GLTextItem object at 0x7fe2a521bdc0>.
Error while drawing item <visualization.GLTextItem object at 0x7fe2a521b550>.
Error while drawing item <visualization.GLTextItem object at 0x7fe2a521ba60>.
Error while drawing item <visualization.GLTextItem object at 0x7fe2a521baf0>.
Error while drawing item <visualization.GLTextItem object at 0x7fe2a521bdc0>.


/home/daniel/.local/lib/python3.8/site-packages/pyqtgraph/opengl/GLViewWidget.py:260: RuntimeWarning: 
Traceback (most recent call last):
  File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/daniel/.local/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/daniel/.local/lib/python3.8/site-packages/traitlets/config/application.py", line 976, in launch_instance
    app.start()
  File "/home/daniel/.local/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/home/daniel/.local/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 215, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.8/asyncio/base_events.py", line 570, in run_forever
    self._run_once()
  File "/usr/lib/python3.8

KeyboardInterrupt: 

# Problem 2 (c)

In [8]:
Tcam = tr.se3(tr.rotz(np.pi/2.0))
T3cam_in_0 = T3_in_0 @ Tcam

T3cam_in_2 = tr.inv(T2_in_0) @ T3cam_in_0

# printing the results
print("new transform of rotated camera frame 3 relative to base frame:")
print(T3cam_in_0)

print("new transform of rotated camera frame 3 relative to frame 2:")
print(T3cam_in_2)

new transform of rotated camera frame 3 relative to base frame:
[[ 1.000000e+00  6.123234e-17  0.000000e+00 -5.000000e-01]
 [ 6.123234e-17 -1.000000e+00  0.000000e+00  1.500000e+00]
 [ 0.000000e+00  0.000000e+00 -1.000000e+00  3.000000e+00]
 [ 0.000000e+00  0.000000e+00  0.000000e+00  1.000000e+00]]
new transform of rotated camera frame 3 relative to frame 2:
[[ 1.000000e+00  6.123234e-17  0.000000e+00  0.000000e+00]
 [ 6.123234e-17 -1.000000e+00  0.000000e+00  0.000000e+00]
 [ 0.000000e+00  0.000000e+00 -1.000000e+00  1.900000e+00]
 [ 0.000000e+00  0.000000e+00  0.000000e+00  1.000000e+00]]


# Problem 2 (d)-i

In [9]:
# define a function that takes in DH parameters (including q) and returns A 
def get_A(dh):
    th = dh[0]
    d = dh[1]
    a = dh[2]
    alpha = dh[3]

    A = tr.se3(R=tr.rotz(th))\
        @tr.se3(p = np.array([0, 0, d]))\
        @tr.se3(p = np.array([a, 0, 0]))\
        @tr.se3(R=tr.rotx(alpha))             

    return A 

# for the following two test cases, evaluate A1 and A2 and plot them 
# (showing the robot in two different configurations)
q_test = [[0.0, 0.0], [1.2, 0.60]]

viz = VizScene()

for i in range(2):
    dh_params = [[q_test[i][0]+np.pi/2.0, 0.0, 0.2, np.pi/2.0],
                 [0, q_test[i][1]+0.30, 0, 0]]
    A1 = get_A(dh_params[0])
    A2 = get_A(dh_params[1])

    viz.add_frame(np.eye(4), label='world')
    viz.add_frame(A1, label='A1_config_'+str(i))
    viz.add_frame(A1 @ A2, label='A2_config_'+str(i))

time_to_run = 30
refresh_rate = 60

for i in range(refresh_rate * time_to_run):
    viz.update()
    time.sleep(1.0/refresh_rate)
    
viz.close_viz()

/home/daniel/.local/lib/python3.8/site-packages/pyqtgraph/opengl/GLViewWidget.py:260: RuntimeWarning: 
Traceback (most recent call last):
  File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/daniel/.local/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/daniel/.local/lib/python3.8/site-packages/traitlets/config/application.py", line 976, in launch_instance
    app.start()
  File "/home/daniel/.local/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/home/daniel/.local/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 215, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.8/asyncio/base_events.py", line 570, in run_forever
    self._run_once()
  File "/usr/lib/python3.8

Error while drawing item <visualization.GLTextItem object at 0x7fe2a519cc10>.
Error while drawing item <visualization.GLTextItem object at 0x7fe2a519c820>.
Error while drawing item <visualization.GLTextItem object at 0x7fe2a519c3a0>.
Error while drawing item <visualization.GLTextItem object at 0x7fe2a51dff70>.
Error while drawing item <visualization.GLTextItem object at 0x7fe2a51bc310>.
Error while drawing item <visualization.GLTextItem object at 0x7fe2a51bc3a0>.
Error while drawing item <visualization.GLTextItem object at 0x7fe2a519cc10>.
Error while drawing item <visualization.GLTextItem object at 0x7fe2a519c820>.
Error while drawing item <visualization.GLTextItem object at 0x7fe2a519c3a0>.
Error while drawing item <visualization.GLTextItem object at 0x7fe2a51dff70>.
Error while drawing item <visualization.GLTextItem object at 0x7fe2a51bc310>.
Error while drawing item <visualization.GLTextItem object at 0x7fe2a51bc3a0>.


KeyboardInterrupt: 